In [1]:
import pandas as pd
import numpy as np

In [2]:
# collections data
csv_file = "resources/Mia_objects_raw.csv"

In [3]:
df = pd.read_csv(csv_file, index_col='Unnamed: 0')
df.head()

,accession_number,artist,classification,continent,country,creditline,culture,dated,department,id,life_date,medium,nationality,object_name,provenance,room,style,title
0,10.1,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c.1888-89,Prints and Drawings,0,"English, 1833 - 1911","Pen and ink, brush and wash over graphite",English,Drawing,"[Art dealer, London, acquired from ""an old hou...",Not on View,19th century,"Air, from the series The Four Elements"
1,10.2,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c. 1888-89,Prints and Drawings,1,"English, 1833 - 1911","Pen and ink, brush and wash over graphite",English,Drawing,"[Art dealer, London, acquired from ""an old hou...",Not on View,19th century,"Earth, from the series The Four Elements"
2,10.3,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c. 1888-89,Prints and Drawings,2,"English, 1833 - 1911","Pen and ink, brush and wash over graphite",English,Drawing,"[Art dealer, London, acquired from ""an old hou...",G352,19th century,"Fire, from the series The Four Elements"
3,10.4,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c. 1888-89,Prints and Drawings,3,"English, 1833 - 1911","Pen and ink, brush and wash over graphite",English,Drawing,"[Art dealer, London, acquired from ""an old hou...",Not on View,19th century,"Water, from the series The Four Elements"
4,13.29,Walter Shirlaw,Drawings,North America,United States,Gift of Mrs. Florence M. Shirlaw,NaN,19th century,Prints and Drawings,4,"American, 1838 - 1909",Watercolor,American,Drawing,NaN,Not on View,19th century,Montana Indian Reservation I


In [4]:
# strip "P" & "p" from prints & drawings
df['accession_number'] = df['accession_number'].str.lstrip('P.')
df['accession_number'] = df['accession_number'].str.lstrip('p.')

# split out first half of accession number
df['accession_year'] = df['accession_number'].str.split('.', n=1, expand=True)[0]

# Acession Number Cleaning

### RBL/RB/X
Adding columns for unknown accession letters

In [5]:
# add 'RBL' column
df['RBL'] = df['accession_year'].str.extract(r'(RBL)', expand=True)
# add 'RB' column
df['RB'] = df['accession_year'].str.extract(r'(RB)', expand=True)
# add 'X' column
df['X'] = df['accession_year'].str.extract(r'(^X)', expand=True)

### L
Adding column for loaned objects

In [6]:
# add loan column
df['loan'] = df['accession_year'].str.extract(r'(^L)', expand=True)

Stripping letters from accession_year

In [7]:
# strip 'RBL' from accession_year column (strips L, RB & RBL)
df['accession_year'] = df['accession_year'].str.lstrip('RBL')

# strip "X" from accession_year column
df['accession_year'] = df['accession_year'].str.lstrip('X')

### Filter out invalid years

Filter dataframe three ways:
- Filter out accession years longer than 4 digits (years go up to 4 digits e.g. 2019)
- Filter out accession numbers shorter than 4 digits (standard accession numbers are in format YY.#)
- Use groupby to filter out any accession years with less than two entries

In [8]:
# replace all empty cells with None
df = df.replace('', None)

In [9]:
# filter dataframe for accession years longer than 4 digits
df_1 = df[df['accession_year'].map(lambda x: len(x) < 5)]
df_2 = df[df['accession_year'].map(lambda x: len(x) > 4)]

In [10]:
# filter dataframe for accession numbers shorter than 4 digits
df_3 = df_1[df_1['accession_number'].map(lambda x: len(x) > 3)]
df_4 = df_1[df_1['accession_number'].map(lambda x: len(x) < 4)]

In [11]:
# Use groupby to filter out any accession years with less than 30 entries
df_grouped = df_3.groupby('accession_year').filter(lambda x: len(x) > 30)
df_remainder = df_3.groupby('accession_year').filter(lambda x: len(x) <= 30)

In [12]:
# combine df_2, df_4, and df_remainder into one larger dataframe
df_remainder = df_remainder.append([df_2,df_4])
df_remainder.creditline.value_counts().head(10)

The Minnich Collection\r\nThe Ethel Morrison Van Derlip Fund, 1966       7573
The William M. Ladd Collection\r\nGift of Herschel V. Jones, 1916        4962
Gift of Mrs. Charles C. Bovey, 1924                                      3133
Gift of George A. Goddard, 1919                                           614
Gift of H. V. Jones                                                       518
The William M. Ladd Collection\r\nGift of Herschel V. Jones, 1916\r\n     325
Gift of Mrs. George P. Douglas, 1946                                      267
Gift of Herschel V. Jones, 1926                                           231
Gift of Mrs. George P. Douglas, 1929                                      212
Gift of Mrs.C.C.Bovey, 1924                                               163
Name: creditline, dtype: int64

### Pull Accession Year from Creditline
Using creditline info, update accession year when possible for objects filtered out of dataframe with invalid accession years

- Pull unique entries from remainder dataframe
- Update accession year using last 4 digits of creditlines
- Re-run data filtering code and check for any manual updates

In [13]:
# pull value counts into dataframe/series
creditlines = df_remainder.creditline.unique()
creditlines_df = pd.DataFrame(creditlines, columns=['creditline'])

In [14]:
# convert all columns to string to fix float type error
creditlines_df['creditline'] = creditlines_df['creditline'].apply(lambda x: str(x))
# strip /r/n from the end of any creditlines
creditlines_df['creditline_new'] = creditlines_df['creditline'].apply(lambda x: x.rstrip('\r\n'))

In [15]:
# add column with last 4 digits of each creditline
creditlines_df['year'] = creditlines_df['creditline_new'].apply(lambda x: x[-4:])

#check to see if year column is numerical
creditlines_df['alpha'] = creditlines_df['year'].apply(lambda x: x.isdigit())
creditlines_df.head()

,creditline,creditline_new,year,alpha
0,"Gift of Mrs. C. J. Martin, in memory of Charle...","Gift of Mrs. C. J. Martin, in memory of Charle...",rtin,False
1,The William Hood Dunwoody Fund,The William Hood Dunwoody Fund,Fund,False
2,Gift of Mr. and Mrs. E.D. Brooks,Gift of Mr. and Mrs. E.D. Brooks,ooks,False
3,Lent by E. H. Hewitt,Lent by E. H. Hewitt,witt,False
4,Gift of M. Knoedler & Co.,Gift of M. Knoedler & Co.,Co.,False


In [16]:
# pull out only creditlines with years at the end
creditlines_year = creditlines_df[creditlines_df['alpha']==True]

# strip 19 from left side of year
creditlines_year['year'] = creditlines_year['year'].map(lambda x: x[-2:])

#drop alpha column from dataframe
creditlines_year = creditlines_year.drop(['alpha','creditline_new'], axis=1)

creditlines_year.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,creditline,year
16,The William M. Ladd Collection\r\nGift of Hers...,16
26,"Gift of Mrs. Charles C. Bovey, 1924",24
29,The Minnich Collection\r\nThe Ethel Morrison V...,66
30,The Minnich Collection\r\n\nThe Ethel Morrison...,66
31,"Gift of Herschel V. Jones, 1925",25


In [17]:
# merge year column to existing dataframe
df = df.merge(creditlines_year, how='left', on='creditline')

In [18]:
# fill nan values in year column using accession_year column
df['year'] = df['year'].fillna(df['accession_year'])

# drop/rename columns so only one 'accession_year' column
df.drop('accession_year',inplace=True,axis=1)
df.rename(columns={'year':'accession_year'}, inplace=True)
df.head()

,accession_number,artist,classification,continent,country,creditline,culture,dated,department,id,...,object_name,provenance,room,style,title,RBL,RB,X,loan,accession_year
0,10.1,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c.1888-89,Prints and Drawings,0,...,Drawing,"[Art dealer, London, acquired from ""an old hou...",Not on View,19th century,"Air, from the series The Four Elements",NaN,NaN,NaN,NaN,10
1,10.2,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c. 1888-89,Prints and Drawings,1,...,Drawing,"[Art dealer, London, acquired from ""an old hou...",Not on View,19th century,"Earth, from the series The Four Elements",NaN,NaN,NaN,NaN,10
2,10.3,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c. 1888-89,Prints and Drawings,2,...,Drawing,"[Art dealer, London, acquired from ""an old hou...",G352,19th century,"Fire, from the series The Four Elements",NaN,NaN,NaN,NaN,10
3,10.4,Frederick G. Smith; Artist: Formerly attribute...,Drawings,Europe,England,"Gift of Mrs. C. J. Martin, in memory of Charle...",NaN,c. 1888-89,Prints and Drawings,3,...,Drawing,"[Art dealer, London, acquired from ""an old hou...",Not on View,19th century,"Water, from the series The Four Elements",NaN,NaN,NaN,NaN,10
4,13.29,Walter Shirlaw,Drawings,North America,United States,Gift of Mrs. Florence M. Shirlaw,NaN,19th century,Prints and Drawings,4,...,Drawing,NaN,Not on View,19th century,Montana Indian Reservation I,NaN,NaN,NaN,NaN,13


### Manually Check and Update Remaining Creditlines

Using code below, pull remainder creditlines and manually change values using .loc

In [19]:
# rerun filters on updated dataframe
# filter dataframe for accession years longer than 4 digits
df_1 = df[df['accession_year'].map(lambda x: len(x) < 5)]
df_2 = df[df['accession_year'].map(lambda x: len(x) > 4)]
# filter dataframe for accession numbers shorter than 4 digits
df_3 = df_1[df_1['accession_number'].map(lambda x: len(x) > 3)]
df_4 = df_1[df_1['accession_number'].map(lambda x: len(x) < 4)]
# Use groupby to filter out any accession years with less than 30 entries
df_grouped = df_3.groupby('accession_year').filter(lambda x: len(x) > 30)
df_remainder = df_3.groupby('accession_year').filter(lambda x: len(x) <= 30)
# combine df_2, df_4, and df_remainder into one larger dataframe
df_remainder = df_remainder.append([df_2,df_4])
df_remainder.creditline.value_counts().head(15)

The William M. Ladd Collection\r\nGift of Herschel V. Jones, 1916    872
Gift of H. V. Jones                                                  518
The Minnich Collection\r\nThe Ethel Morrison Van Derlip Fund         114
The Ethel Morrison Van Derlip Fund                                    55
Gift of F.N. Edmonds                                                  52
Gift of Frederick B. Wells                                            38
Gift of Mrs. Carl W. Jones in Memory of Her Husband                   38
Gift of Mrs. Darwin R. Martin                                         38
The William Hood Dunwoody Fund                                        31
Gift of the Estate of Dorothy Millett Lindeke                         30
The William Hood Dunwoody Fund, 1915                                  28
Gift of Bruce B. Dayton                                               24
Gift of Ethel Morrison Van DerLip, 1916                               22
The John R. Van Derlip Print Fund                  

In [20]:
# Update remaining entries
df.loc[(df['creditline'] == 'The Miscellaneous Works of Art Purchase Fund, 1952 (2nd Biennial)'), 
            'accession_year'] = '52'
df.loc[(df['creditline'] == 'The William M. Ladd Collection\r\nGift of Herschel V. Jones, 1916t'), 
            'accession_year'] = '16'
df.loc[(df['creditline'] == 'The William Hood Dunwoody Fund, 1954 (Biennial)'), 
            'accession_year'] = '54'
df.loc[(df['creditline'] == 'Gift of Frederick B. Wells, 1943\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n++++++++++++++++++++++++++++++++++++++++'), 
            'accession_year'] = '43'
df.loc[(df['creditline'] == 'Gift of Mr. Davis, 1948?'), 
            'accession_year'] = '48'
df.loc[(df['creditline'] == 'The Ethel Morrison Van Derlip Fund, 1967 (?)'), 
            'accession_year'] = '67'
df.loc[(df['creditline'] == 'The William M. Ladd Collection, 1916\r\nGift of Herschel V. Jones'), 
            'accession_year'] = '16'
df.loc[(df['creditline'] == 'The William M. Ladd Collection\r\nGift of Herschel V. Jones, 1916s'), 
            'accession_year'] = '16'
df.loc[(df['creditline'] == 'The John De Laittre Memorial Collection, Gift of Mrs. Horace Ropes, 1940 (?)'), 
            'accession_year'] = '40'
df.loc[(df['creditline'] == 'Gift of Herschel V. Jones (1968?)'), 
            'accession_year'] = '68'
df.loc[(df['creditline'] == 'Carl W. Jones Memorial Fund, 1957 and The Miscellaneous Works of Art Purchase Fund'), 
            'accession_year'] = '57'

### Objects with no accession year info

Remainder dataframe objects that do not have enough information to update accession year: 2295
- Will leave as is and filter out of any visualizations

In [23]:
# rerun filters on updated dataframe
# filter dataframe for accession years longer than 4 digits
df_1 = df[df['accession_year'].map(lambda x: len(x) < 5)]
df_2 = df[df['accession_year'].map(lambda x: len(x) > 4)]

# filter dataframe for accession years equal to three digits
df_3 = df_1[df_1['accession_year'].map(lambda x: len(x) != 3)]
df_4 = df_1[df_1['accession_year'].map(lambda x: len(x) == 3)]

# Use groupby to filter out any accession years with less than 1 entry
df_grouped = df_3.groupby('accession_year').filter(lambda x: len(x) > 1)
df_remainder = df_3.groupby('accession_year').filter(lambda x: len(x) <= 1)

# combine df_2, df_4, and df_remainder into one larger dataframe
df_remainder = df_remainder.append([df_2,df_4])
len(df_remainder)

2295

In [27]:
# New file for visualizations from objects with updated accession years
output_datafile = 'resources/Mia_objects_accession_year.csv'
df_grouped.to_csv(output_datafile, encoding='utf-8')

In [ ]:
df